In [1]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_openai import OpenAIEmbeddings


In [2]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [3]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH99-memory_test")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH99-memory_test


In [4]:
loader1 = TextLoader("./data/text.txt", encoding="utf-8")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=0)
split_doc1 = loader1.load_and_split(text_splitter)
print(type(split_doc1))
print(split_doc1[1])
print(len(split_doc1))  

<class 'list'>
page_content='2021년 5월에는 상속 절차가 진행돼 홍라희 리움미술관 명예관장에게 9분의 3, 이재용 삼성전자 회장·이부진 호텔신라 사장·이서현 삼성물산 사장에게 각각 9분의 2씩 지분이 나눠졌다. 삼성 일가는 약 4년간 이 집을 보유하다가 올 초부터 매각 절차를 진행해 6월에 거래를 마무리했다.
해당 주택은 ‘삼성가족타운’으로 불리는 이태원 언덕길 인근에 자리하고 있다. 이번 거래가는 토지 기준 3.3㎡(평)당 약 7000만원 수준으로, 2010년 당시 3.3㎡당 2500만원에 거래된 것과 비교해 약 175% 오른 셈이다.
1984년생인 강나연 회장은 에너지·철강 트레이딩 전문기업인 태화홀딩스의 창업자다. 2013년에 설립된 태화홀딩스는 러시아, 인도네시아, 호주 등에서 석탄, 펫콕, 합금철 등 에너지 및 철강 원자재를 수입해 아시아 시장에 공급하고 있다.
태화홀딩스의 최근 3년 매출은 2022년 2733억원, 2023년 3376억원, 2024년 4055억원으로 꾸준히 성장세를 보이고 있다.
한편 삼성 일가는 지난해 10월에도 이 선대회장이 남긴 또 다른 이태원 단독주택을 매각한 바 있다.16일 서울 여의도 한국거래소 전광판에 코스피 종가가 표시돼 있다.' metadata={'source': './data/text.txt'}
85


In [5]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

model_name = "intfloat/multilingual-e5-large-instruct"
# model_name = "intfloat/multilingual-e5-large"

hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device": "cuda"},  # cuda, cpu
    encode_kwargs={"normalize_embeddings": True},
)

In [6]:
db = FAISS.from_documents(documents=split_doc1, embedding=hf_embeddings)

In [ ]:
import torch
import gc

del db 
del hf_embeddings               # Delete embedding object
gc.collect()                   # Garbage collect
torch.cuda.empty_cache()       # Empty CUDA cache